In [1]:
import pandas as pd
import numpy as np

In [2]:
#load data
data_df = pd.read_csv("../db/stock_data_with_forward_perf.csv")
print(data_df.columns)
data_df.head()

Index(['name', 'ticker', 'Quarter end', 'Shares', 'Shares split adjusted',
       'Split factor', 'Assets', 'Current Assets', 'Liabilities',
       'Current Liabilities', 'Shareholders equity',
       'Non-controlling interest', 'Preferred equity',
       'Goodwill & intangibles', 'Long-term debt', 'Revenue', 'Earnings',
       'Earnings available for common stockholders', 'EPS basic',
       'EPS diluted', 'Dividend per share', 'Cash from operating activities',
       'Cash from investing activities', 'Cash from financing activities',
       'Cash change during period', 'Cash at end of period',
       'Capital expenditures', 'Price', 'Price high', 'Price low', 'ROE',
       'ROA', 'Book value of equity per share', 'P/B ratio', 'P/E ratio',
       'Cumulative dividends per share', 'Dividend payout ratio',
       'Long-term debt to equity ratio', 'Equity to assets ratio',
       'Net margin', 'Asset turnover', 'Free cash flow per share',
       'Current ratio', 'mktcap_revenue_value', '

,name,ticker,Quarter end,Shares,Shares split adjusted,Split factor,Assets,Current Assets,Liabilities,Current Liabilities,...,two_quarter_return,one_year_return,two_year_return,three_year_return,four_year_return,five_year_return,seven_year_return,ten_year_return,twelve_year_return,fifteen_year_return
0,Alcoa Corporation,AA,2018-03-31,482832111,482832111,1,18219000000,5895000000,12937000000,2802000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alcoa Corporation,AA,2017-12-31,482772252,482772252,1,18718000000,6378000000,13794000000,2824000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Alcoa Corporation,AA,2017-09-30,481324177,481324177,1,19237000000,6148000000,13276000000,2677000000,...,9.329564,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Alcoa Corporation,AA,2017-06-30,441030999,441030999,1,19106000000,6033000000,13353000000,2658000000,...,2.460317,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Alcoa Corporation,AA,2017-03-31,440826482,440826482,1,20157000000,6710000000,14662000000,2587000000,...,2.392539,10.948905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
#specify the features(measures) and target (label) to be used

import collections

#get features data to be used. Change for comparing for selected parameters
parameters_columns = ['EPS basic','ROE','ROA', 'P/B ratio', 'P/E ratio', 'Dividend payout ratio',
                   'Long-term debt to equity ratio', 'Equity to assets ratio', 'Free cash flow per share',
                   'Current ratio', 'mktcap_revenue_value', 'mktcap_free_cash_flow_value',
                   'mktcap_cash_value', 'cash_oper_gt_earnings_value',
                   'entvalue_earnings_value', 'marketcap_bookvalue_value']

#set target values
target_values = ['no','yes']


#change for selected different investment period
selected_target_column = 'one_year_return'

#set calculated values whether a stock is a buy (1) or not (2). Standard  used is 5% for each year, so 5% for a year, 15% for 3 years and 25 for 5 years
data_df.loc[data_df[selected_target_column] >= 8.0, 'calc_buystock'] = 1
data_df.loc[data_df[selected_target_column] < 8.0, 'calc_buystock'] = 0


#separate data into training and training.
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)
#using 1st qtr 2016, change the ending train date according to investment period (selected_target_column). 
#It should that much time period behind the target_start_date to avoid using future data
target_start_date = '2009-01-01'
target_end_date = '2009-03-31'
ending_train_date = '2007-12-31'

test_data_df = data_df.loc[(data_df['Quarter end'] >= target_start_date)&(data_df['Quarter end']<=target_end_date)]
y_test = test_data_df['calc_buystock'].values

print(f"Counts for calculated buy for test data: {collections.Counter(y_test)}")
X_test = test_data_df[parameters_columns].values

train_data_df = data_df.loc[data_df['Quarter end'] <= ending_train_date]
y_train = train_data_df['calc_buystock'].values
X_train = train_data_df[parameters_columns].values
print(f"Counts for calculated buy for train data: {collections.Counter(y_train)}")

Counts for calculated buy for test data: Counter({1.0: 563, 0.0: 99})
Counts for calculated buy for train data: Counter({1.0: 18376, 0.0: 13158})


In [7]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.7175226586102719

In [5]:
sorted(zip(rf.feature_importances_, parameters_columns), reverse=True)

[(0.10812225843065226, 'Free cash flow per share'),
 (0.09625263008443703, 'Equity to assets ratio'),
 (0.09605235346499415, 'P/B ratio'),
 (0.08604934032985295, 'ROA'),
 (0.08508226809770825, 'EPS basic'),
 (0.08346053147582622, 'Current ratio'),
 (0.07844038241143192, 'Long-term debt to equity ratio'),
 (0.07697201094441308, 'ROE'),
 (0.07583808724059787, 'P/E ratio'),
 (0.06313768675448474, 'Dividend payout ratio'),
 (0.059149986819334345, 'cash_oper_gt_earnings_value'),
 (0.04502981022420773, 'mktcap_cash_value'),
 (0.0412204286230208, 'mktcap_revenue_value'),
 (0.003241789494316036, 'marketcap_bookvalue_value'),
 (0.0010952846359428819, 'mktcap_free_cash_flow_value'),
 (0.0008551509687794504, 'entvalue_earnings_value')]

In [ ]:
data_df.head()

In [ ]:
import pickle

# Save to file in the current working directory
pkl_filename = "RFmodel_3yr_16F.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(rf, file)

In [ ]:
from sklearn.metrics import classification_report
predictions = rf.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=["no", "yes"]))

In [ ]:
predictions

In [ ]:
test_data_df['predicted_buys'] = predictions
test_data_df.head()

In [ ]:
test_data_df.to_csv('stock_predictions_1qtr2009.csv')